## Preambule

In [1]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import pandas as pd
import json
import plotly

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameters

In [2]:
rules = ['PCC', 'ECPC', 'AP']
rulecolors = ['goldenrod', 'tomato', 'forestgreen']
year_of_focus = 2035

## Paths

In [3]:
path_main = Path("K:/Code/effort-sharing/")
path_data = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/")
path_figs = path_main / "Figures" / "Paper_FairShares"

## Read data files

In [4]:
xr_dataread = xr.open_dataset(path_data / "startyear_2021/xr_dataread.nc")
xr_traj_yeari = xr.open_dataset(path_data / ("startyear_2021/Aggregated_files/xr_alloc_"+str(year_of_focus)+"_GHG_incl.nc"))
xr_alloc_yeari = xr.open_dataset(path_data / ("startyear_2021/Aggregated_files/xr_alloc_"+str(year_of_focus)+"_GHG_incl.nc"))
xr_total = xr.open_dataset(path_data / "xr_policyscen.nc")

all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

## Get Cost-optimal output

In [5]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

## Compute fractions

World

In [6]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_World_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss', 'Price|Carbon'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6['Region'] = 'WORLD'
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen = xr.Dataset.from_dataframe(dummy)
xr_scen = xr_scen.reindex(Time = np.arange(1850, 2101))
xr_scen = xr_scen.interpolate_na(dim="Time", method="linear")

Regional

In [7]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss', 'Price|Carbon'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

R-10 regions

In [8]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_R10/AR6_Scenarios_Database_R10_regions_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss', 'Price|Carbon'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_R10/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r10 = xr.Dataset.from_dataframe(dummy)
xr_scen_r10 = xr_scen_r10.reindex(Time = np.arange(1850, 2101))
xr_scen_r10 = xr_scen_r10.interpolate_na(dim="Time", method="linear")

In [9]:
df_conv = pd.read_excel("X:/user/dekkerm/Data/AR6_regionclasses.xlsx", sheet_name = 'Sheet1')[['ISO', 'region_ar6_10']]
regs = np.array(df_conv.region_ar6_10)
regs[regs == 'Africa'] =            'R10AFRICA'
regs[regs == 'Eastern Asia'] =      'R10CHINA+'
regs[regs == 'Europe'] =            'R10EUROPE'
regs[regs == 'Southern Asia'] =     'R10INDIA+'
regs[regs == 'Latin America and Caribbean'] = 'R10LATIN_AM'
regs[regs == 'Middle East'] =       'R10MIDDLE_EAST'
regs[regs == 'North America'] =     'R10NORTH_AM'
regs[regs == 'Asia-Pacific Developed'] = 'R10PAC_OECD'
regs[regs == 'Eurasia'] =           'R10REF_ECON'
regs[regs == 'South-East Asia and developing Pacific'] = 'R10REST_ASIA'
df_conv['region_ar6_10'] = regs

In [10]:
native_regions = np.array(xr_scen_r.Region)
xrs = []
for cty in all_countries_iso:
    if cty not in native_regions:
        which_group = np.array(df_conv[df_conv.ISO == cty].region_ar6_10)[0]
        group_members = np.array(df_conv[df_conv.region_ar6_10 == which_group].ISO)
        emis_total = float(xr_dataread.GHG_hist.sel(Region=np.intersect1d(group_members,all_countries_iso), Time=2021).sum(dim='Region'))
        emis_frac = xr_dataread.GHG_hist.sel(Region=cty, Time=2021) / emis_total
        xrry = xr.merge([xr_scen_r10.sel(Region=which_group, Variable=['Emissions|Kyoto Gases'])*emis_frac,
                         xr_scen_r10.sel(Region=which_group, Variable=['Price|Carbon'])]).expand_dims({'Region': [cty]})
        xrs.append(xrry)
xr_scen_infilled_r = xr.concat(xrs, dim='Region')

Concatenate

In [11]:
xr_scen_all = xr.merge([xr_scen, xr_scen_r, xr_scen_infilled_r])

## Data for plots

In [12]:
settings_default = {'Temperature': 1.6,
                   'Risk': 0.5,
                   'NegEmis': 0.5,
                   'NonCO2red': 0.5,
                   'Timing': "Immediate",
                   'Convergence_year': 2050,
                   'Scenario': 'SSP2',
                   'Discount_factor': 0,
                   'Historical_startyear': 1990,}
                   #'Capability_threshold': "Th",
                   #'RCI_weight': "Half"}
settings_ranges = {'Temperature': [1.6, 2.0],
                   'Risk': [0.5, 0.33],
                   'Convergence_year': [2050, 2080],
                    'NonCO2red':[0.33, 0.5, 0.67],
                   'Timing': ["Immediate"],
                   }
settings_ranges2 = {'Temperature': [1.6, 2.0],
                   'Risk': [0.5, 0.33],
                   'Convergence_year': [2050, 2080],
                    'NonCO2red':[0.33, 0.5, 0.67],
                   'Timing': ["Immediate"],
                   }
                   #'Capability_threshold': "Th",
                   #'RCI_weight': "Half"}

In [13]:
ndcs = xr_dataread.GHG_ndc.mean(dim=['Conditionality', 'Ambition'])
data = xr_traj_yeari.sel(**settings_ranges)[['ECPC', 'AP', 'PCC']]
data_max = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').max(dim=list(settings_default.keys())+['variable2'])
data = xr_traj_yeari.sel(**settings_default)[['ECPC', 'AP', 'PCC']]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim='variable2')

In [14]:
print("Preferences on default settings")
print('-----')
data = xr_traj_yeari.sel(**settings_default)[['ECPC', 'AP', 'PCC']]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim='variable2')
argmax_default = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').fillna(-9e99).argmax(dim='variable2')

data = xr_traj_yeari.sel(**settings_ranges)[['ECPC', 'AP', 'PCC']]
data_max = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').max(dim=list(settings_default.keys())+['variable2'])
argmax_maximum = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').fillna(-9e99).argmax(dim=list(settings_default.keys())+['variable2'])
x = xr.concat([data.ECPC, data.AP.expand_dims({'Convergence_year': [2050, 2080]}), data.PCC], dim='variable2').fillna(-9e99)
argmax_max = []
maxs = x.max(dim=list(settings_default.keys())+['variable2'])
for cty in x.Region:
    mx = float(maxs.sel(Region=cty).values)
    wh = np.where(x.sel(Region=cty).values == mx)
    wh = np.array(wh)[:, 0]
    argmax_max.append(wh[0])
argmax_max = np.array(argmax_max)

df_c = []
df_r_d = []
df_r_m = []
for cty_i, cty in enumerate(all_countries_iso):
    df_c.append(all_countries_names[cty_i])
    df_r_d.append(['ECPC', 'AP', 'PCC'][argmax_default.sel(Region=cty).values])
    df_r_m.append(['ECPC', 'AP', 'PCC'][argmax_max[cty_i]])
df_dict = {'Region': df_c, 'Default': df_r_d, 'Maximum': df_r_m}
df = pd.DataFrame(df_dict)
df.to_csv(path_data / "preferences_default_max.csv")

Preferences on default settings
-----


In [15]:
xr_policyscen = xr.open_dataset(path_data / "xr_policyscen.nc")
curpols = xr_policyscen.CurPol.sel(Time=year_of_focus).mean(dim='Model')
nzs = xr_policyscen.NetZero.sel(Time=year_of_focus).mean(dim='Model')

In [16]:
path_ctygroups = "X:/user/dekkerm/Data/" + "UNFCCC_Parties_Groups_noeu.xlsx"
df = pd.read_excel(path_ctygroups, sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
countries_name = np.array(df["Name"])
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [17]:
costopts = xr_scen_all.sel(Time=year_of_focus, Variable='Emissions|Kyoto Gases').mean(dim='ModelScenario')

## Gaps

In [18]:
nec = xr_dataread.GHG_globe.sel(Time=year_of_focus, Temperature=1.6, Risk=0.5, NonCO2red=0.5, NegEmis=0.5, Timing='Immediate').values
gap_default = float(data_def.sel(Region=countries_iso).sum() - nec)
gap_default_perc = float((data_def.sel(Region=countries_iso).sum() - nec)/nec)

gap_max = float(data_max.sel(Region=countries_iso).sum() - nec)
gap_max_perc = float((data_max.sel(Region=countries_iso).sum() - nec)/nec)

## Gaps

In [19]:
wd = 0.4
q0 = 0.1
q1 = 0.9

In [20]:
co = costopts.Value
co_0 = xr_scen_all.sel(Time=year_of_focus, Variable='Emissions|Kyoto Gases').quantile(q0, dim='ModelScenario')
co_1 = xr_scen_all.sel(Time=year_of_focus, Variable='Emissions|Kyoto Gases').quantile(q1, dim='ModelScenario')
ndc = xr_dataread.GHG_ndc.mean(dim='Ambition').min(dim='Conditionality')
ndc = ndc.where(ndc.Region!='ZMB', np.nan)
ndc = ndc.where(ndc.Region!='COM', np.nan)
fair = xr_alloc_yeari.sel(Temperature=1.6,
                                Risk=0.5,
                                Timing='Immediate',
                                NonCO2red=0.5,
                                NegEmis=0.5,
                                Scenario='SSP2',
                                Convergence_year=2050,
                                Discount_factor=0,
                                Historical_startyear=1990)[['PCC', 'AP', 'ECPC']]
fair_0 = xr_alloc_yeari.sel(Temperature=1.6,
                            Risk=0.5,
                            Convergence_year=[2050, 2080])[['PCC', 'AP', 'ECPC']].quantile(0, dim=['Convergence_year', 'NegEmis', 'NonCO2red', 'Timing', 'Scenario', 'Discount_factor', 'Historical_startyear'])
fair_1 = xr_alloc_yeari.sel(Temperature=1.6,
                            Risk=0.5,
                            Convergence_year=[2050, 2080])[['PCC', 'AP', 'ECPC']].quantile(1, dim=['Convergence_year', 'NegEmis', 'NonCO2red', 'Timing', 'Scenario', 'Discount_factor', 'Historical_startyear'])

totaldev = fair - co # If positive, receive funds, if negative, pay funds (or partially domestic)
ndcdev = ndc - co # If positive, more domestic action needed, and vice versa
grey = (ndcdev - totaldev.where(totaldev > 0, 0)).where(ndcdev - totaldev.where(totaldev > 0, 0) > 0, 0)
red = -totaldev.where(totaldev < 0, 0)
green = totaldev.where(totaldev > 0, 0)
grey_super = (co-ndc).where(fair < co, 0)#-(ndcdev - totaldev.where(totaldev > 0, 0)).where(ndcdev - totaldev.where(totaldev > 0, 0) < 0, 0)
grey_super = grey_super.where(grey_super > 0, 0)

for cty in group_eu: # To avoid double counting!
    grey.loc[{'Region': cty}] = np.nan
    red.loc[{'Region': cty}] = np.nan
    green.loc[{'Region': cty}] = np.nan
    grey_super.loc[{'Region': cty}] = np.nan


k:\Environments\MyEnv\Lib\site-packages\numpy\lib\nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
k:\Environments\MyEnv\Lib\site-packages\numpy\lib\nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [21]:
costopt_mt = costopts.Value
ndc_est_mt = xr_dataread.GHG_ndc.mean(dim='Ambition').min(dim='Conditionality')
red_mt = xr_alloc_yeari.sel(Temperature=1.6,
                                 Risk=0.5,
                                 Timing='Immediate',
                                 NonCO2red=0.5,
                                 NegEmis=0.5,
                                 Scenario='SSP2',
                                 Convergence_year=2050,
                                 Discount_factor=0,
                                 Historical_startyear=1990)

emis_2015 = xr_dataread.GHG_hist.sel(Time=2015)
reductions = 1-red_mt/emis_2015
ndc_est = 1-(ndc_est_mt / emis_2015)
costopt = 1-costopt_mt/emis_2015

red_0 = 1-fair_0/emis_2015
red_1 = 1-fair_1/emis_2015
costopt_0 = 1-co_0/emis_2015
costopt_1 = 1-co_1/emis_2015

def data(reg): # Reduction targets
    return float(ndc_est.sel(Region=reg)), np.array(reductions.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array()), float(costopt.sel(Region=reg))
def data_mt(reg): # Absolute emissions levels
    return float(ndc_est_mt.sel(Region=reg)), np.array(red_mt.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array()), float(costopt_mt.sel(Region=reg))

## Finance plot

In [22]:
col_receive = 'forestgreen'
dx = 0.03
opac = 0.5
def areas(fig, ndc_estimate, reductions_i, costopt_i, row, col, yref=1):
    for i in range(3):
        ymin = yref + -wd*0.3 + [-wd*0.7, 0, wd*0.7][i]
        ymax = yref + wd*0.3+ [-wd*0.7, 0, wd*0.7][i]
        if reductions_i[i] > ndc_estimate:
            if costopt_i < reductions_i[i] and costopt_i > ndc_estimate:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=ndc_estimate+dx, x1=costopt_i-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    name='Increase domestic ambition',
                    showlegend=False,
                    opacity = opac,
                    legendgroup=3,
                    fillcolor='grey',  # Transparent fill color (adjust alpha for transparency)
                )
            elif costopt_i > reductions_i[i]:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=ndc_estimate+dx, x1=reductions_i[i]-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = opac,
                    fillcolor='grey',  # Transparent fill color (adjust alpha for transparency)
                )
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=reductions_i[i]+dx, x1=costopt_i-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = opac,
                    fillcolor=col_receive,  # Transparent fill color (adjust alpha for transparency)
                )
            elif costopt_i < ndc_estimate-0.0425:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=costopt_i+dx, x1=ndc_estimate-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = opac,
                    fillcolor='rosybrown',  # Transparent fill color (adjust alpha for transparency)
                )
        # Pay abroad
        if reductions_i[i] > ndc_estimate:
            if costopt_i < ndc_estimate:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=ndc_estimate+dx, x1=reductions_i[i]-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    name='Pay for mitigation abroad',
                    showlegend=False,
                    opacity = opac,
                    legendgroup=3,
                    fillcolor='tomato',  # Transparent fill color (adjust alpha for transparency)
                )
            elif costopt_i < reductions_i[i]:
                fig.add_shape(
                    row = row, col=col,
                    type='rect',  # Draw a rectangle
                    x0=costopt_i+dx, x1=reductions_i[i]-dx,  # The range of the box (between min and max values)
                    y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                    line=dict(color='black', width=0),  # Border color and width of the box
                    opacity = opac,
                    fillcolor='tomato',  # Transparent fill color (adjust alpha for transparency)
                )
        # Receive from abroad
        if reductions_i[i] < ndc_estimate:
            fig.add_shape(
                row = row, col=col,
                type='rect',  # Draw a rectangle
                x0=reductions_i[i]+dx, x1=ndc_estimate-dx,  # The range of the box (between min and max values)
                y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                line=dict(color='black', width=0),  # Border color and width of the box
                name='Receive mitigation payment from abroad',
                showlegend=False,
                    opacity = opac,
                legendgroup=3,
                fillcolor=col_receive,  # Transparent fill color (adjust alpha for transparency)
            )
            fig.add_shape(
                row = row, col=col,
                type='rect',  # Draw a rectangle
                x0=ndc_estimate+dx, x1=costopt_i-dx,  # The range of the box (between min and max values)
                y0=ymin, y1=ymax,  # Position the box on the y-axis (with some vertical padding)
                line=dict(color='black', width=0),  # Border color and width of the box
                name='Receive mitigation payment from abroad',
                showlegend=False,
                opacity = opac,
                legendgroup=3,
                fillcolor=col_receive,  # Transparent fill color (adjust alpha for transparency)
            )

In [23]:
fig = make_subplots(rows=5, cols=3,
                    specs=[[{'colspan':2}, {}, {}],
                           [{'rowspan': 4, 'colspan':2}, {}, {'rowspan': 4}],
                           [{}, {}, {}],
                           [{}, {}, {}],
                           [{}, {}, {}]],
                    horizontal_spacing = 0., vertical_spacing=0.02)

fig.update_layout(height=1000, width=1000)
fs=20
fig['layout'].update(
    annotations=[dict(x=0.05, y=1.08, text='<b>NDC', xref='paper', yref='paper', showarrow=False, font=dict(color='black', size=18)),
                 dict(x=0.17, y=1.1, text='<b>Cost<br>optimal', xref='paper', yref='paper', showarrow=False, font=dict(color='saddlebrown', size=18)),
                 dict(x=0.45, y=1.1, text="<b>Fair targets<br>according to<br><span style='color: purple;'>ECPC</span>, "
                                            "<span style='color: goldenrod;'>AP</span> and "
                                            "<span style='color: navy;'>PCC</span>",
                                            xref='paper', yref='paper', showarrow=False, font=dict(color='black', size=18)),
                #  dict(x=0.01, y=0.98, text='<b>ECPC', xref='paper', yref='paper', showarrow=False, font=dict(color='purple', size=fs)),
                dict(x=-0.05, y=0.94, text='<b>Example<br>country', xref='paper', yref='paper', align = 'left',showarrow=False, font=dict(color='black', size=16)),
                dict(x=0.27, y=0.98, text='<b>Provide finance', xref='paper', yref='paper', align = 'left',showarrow=False, font=dict(color='darkred', size=16)),
                dict(x=0.29, y=0.91, text='<b>Receive<br>finance', xref='paper', yref='paper', align = 'left',showarrow=False, font=dict(color=col_receive, size=16)),

                 dict(x=0.76, y=1.1, text='<b>Fund<br>domestic<br>mitigation', xref='paper', yref='paper', align='center', showarrow=False, font=dict(color='grey', size=18)),
                 dict(x=0.88, y=1.1, text='<b>Receive<br>finance', xref='paper', yref='paper', showarrow=False, align='center', font=dict(color=col_receive, size=18)),
                 dict(x=1.01, y=1.1, text='<b>Provide<br>finance', xref='paper', yref='paper', showarrow=False, align='center', font=dict(color='tomato', size=18)),
                 dict(x=0.75, y=1.01, text='(Gt CO<sub>2</sub>e)', xref='paper', yref='paper', align='center', showarrow=False, font=dict(color='grey', size=15)),
                 dict(x=0.88, y=1.01, text='(Gt CO<sub>2</sub>e)', xref='paper', yref='paper', showarrow=False, align='center', font=dict(color=col_receive, size=15)),
                 dict(x=1.01, y=1.01, text='(Gt CO<sub>2</sub>e)', xref='paper', yref='paper', showarrow=False, align='center', font=dict(color='tomato', size=15)),
                 dict(x=0.25, y=-0.08, text='Emissions reductions by ' + str(year_of_focus)+' w.r.t. 2015', xref='paper', yref='paper', xanchor='center', showarrow=False, font=dict(color='black', size=15)),
                 
                 ])

for i in range(5):
    fig.add_shape(
        type="line",
        x0=[0.08, 0.22, 0.45, 0.28, 0.28][i], x1=[0.11, 0.20, 0.49, 0.18, 0.18][i],  # Line spans horizontally across the paper
        y0=[1.04, 1.04, 1.01, 0.89, 0.89][i], y1=[0.99, 0.99, 0.99, 0.91, 0.87][i],  # Line fixed at y=5 in subplot data coordinates
        xref="paper",  # Reference x-coordinates relative to the paper
        yref="paper",     # Reference y-axis of the first subplot
        line=dict(color="black", width=2)  # Line style
    )

# ============ #
# Panel (a) Explainer
# ============ #

reg = 'SAU'
if year_of_focus == 2030: dash = 'solid'
else: dash = 'dot'
ndc_estimate, reductions_i, costopt_i = data(reg)
reductions_i[1] = -0.1
reductions_i[0] = -0.5

fig.add_shape(
    type="line",
    col=1, row=1,
    x0=ndc_estimate, x1=ndc_estimate,
    y0=1 + -wd, y1=1 + wd,
    name='NDC',
    legendgroup=2,
    showlegend=False,
    line=dict(color="black", width=5, dash=dash)
)
for i in range(3):
    fig.add_shape(
        type="line",
        col=1, row=1,
        x0=reductions_i[i], x1=reductions_i[i],
        y0=1 + -wd*0.3 + [-wd*0.7, 0, wd*0.7][i], y1=1 + wd*0.3+ [-wd*0.7, 0, wd*0.7][i],
        name=['Per capita convergence', 'Ability to pay', 'Equal cumulative per capita'][i],
        legendgroup=2,
        showlegend=False,
        line=dict(color=["steelblue", 'goldenrod', 'purple'][i], width=5)
    )
areas(fig, ndc_estimate, reductions_i, costopt_i, 1, 1)
fig.add_shape(
    type="line",
    col=1, row=1,
    x0=costopt_i, x1=costopt_i,
    y0=-wd + 1, y1=wd + 1,
    name='Cost optimal',
    legendgroup=2,
    showlegend=False,
    line=dict(color="saddlebrown", width=5)
)

fig.add_shape(
    col=1, row=1,
    type="line",
    x0=0, x1=0,
    y0=0.5, y1=2,
    line=dict(color="black",  dash='dash',width=3)
)
fig.update_yaxes(row=1, col=1, range=(0.5, 1.5), showticklabels=False)

fig.update_layout(
    plot_bgcolor='white',  # Background of the plot area
    paper_bgcolor='white',  # Background of the entire figure
    barmode='group',  # Ensures the bars are grouped
)
fig.update_xaxes(row=1, col=1, range=(-1, 3), showticklabels=False)

# Annotation
for i in range(3):
    fig.add_annotation(text=["1.0", "0", "4.0"][i], x=[0.1, 0.5, 0.9][i], y=0.8, showarrow=False, row=1, col=3, font=dict(size=17, color=['grey',col_receive, 'tomato'][i]))
    fig.add_annotation(text=["0.5", "0.5", "0"][i], x=[0.1, 0.5, 0.9][i], y=0.5, showarrow=False, row=1, col=3, font=dict(size=17, color=['grey', col_receive, 'tomato'][i]))
    fig.add_annotation(text=["0", "1.0", "0"][i], x=[0.1, 0.5, 0.9][i], y=0.2, showarrow=False, row=1, col=3, font=dict(size=17, color=['grey', col_receive, 'tomato'][i]))
fig.update_xaxes(row=1, col=3, range=(0, 1), showticklabels=False)
fig.update_yaxes(row=1, col=3, range=(0, 1), showticklabels=False)

# ============ #
# Actual data
# ============ #

for reg_i, reg in enumerate(['USA', 'EU', 'SAU', 'BRA', 'IDN', 'VNM', 'CHN', 'PAK', 'IND', 'NGA'][::-1]):
    ndc_estimate, reductions_i, costopt_i = data(reg)

    # ACTUAL DATA
    fig.add_shape(
        type="line",
        col=1, row=2,
        x0=ndc_estimate, x1=ndc_estimate,
        y0=reg_i*1.25 + -wd, y1=reg_i*1.25 + wd,
        name='NDC',
        legendgroup=2,
        showlegend=False,
        line=dict(color="black", width=5, dash=dash)
    )

    if np.abs(ndc_estimate - costopt_i) < 0.0425:
        costopt_plot = ndc_estimate - 0.0425
    else: costopt_plot = costopt_i
    fig.add_shape(
        type="line",
        col=1, row=2,
        x0=costopt_plot, x1=costopt_plot,
        y0=-wd + reg_i*1.25 , y1=wd + reg_i*1.25,
        name='Cost optimal',
        legendgroup=2,
        showlegend=False,
        line=dict(color="saddlebrown", width=5)
    )

    for i in range(3):
        fig.add_shape(
            type="line",
            col=1, row=2,
            x0=reductions_i[i], x1=reductions_i[i],
            y0=reg_i*1.25 + -wd*0.3 + [-wd*0.7, 0, wd*0.7][i], y1=reg_i*1.25 + wd*0.3+ [-wd*0.7, 0, wd*0.7][i],
            name=['Per capita convergence', 'Ability to pay', 'Equal cumulative per capita'][i],
            legendgroup=2,
            showlegend=False,
            line=dict(color=["steelblue", 'goldenrod', 'purple'][i], width=5)
        )

    areas(fig, ndc_estimate, reductions_i, costopt_i, 2, 1, yref=reg_i*1.25)
    cty = all_regions_names[all_regions_iso==reg][0].replace(' ', '<br>')
    if reg == 'USA': cty = 'United<br>States'
    fig.add_annotation(x=-0.05, y=reg_i*1.25, align = 'left',
                       text=cty, xref='paper', yref='y4', showarrow=False, font=dict(color='black', size=16))
    
    domestic = np.array(grey.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array())
    receivepay = np.array(green.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array())
    domoverspend = np.array(grey_super.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array())
    payabroad = np.array(red.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array()) - domoverspend # Some payments are done 'domestically' if NDC reductions > CO
    
    d0, d1 = (np.min(domestic)/1e3, np.max(domestic)/1e3)
    r0, r1 = (np.min(receivepay)/1e3, np.max(receivepay)/1e3)
    p0, p1 = (np.min(payabroad)/1e3, np.max(payabroad)/1e3)
    o0, o1 = (np.min(domoverspend)/1e3, np.max(domoverspend)/1e3)

    if p0 == 0: p0 = int(0)
    if r0 == 0: r0 = int(0)
    if d0 == 0: d0 = int(0)
    if o0 == 0: o0 = int(0)

    s_d = f"{d0:.1f}-{d1:.1f}"
    s_r = f"{r0:.1f}-{r1:.1f}"
    s_p = f"{p0:.1f}-{p1:.1f}"
    s_o = f"{o0:.1f}-{o1:.1f}"

    if d0 == d1:
        if d0 != 0:
            s_d = f"{d0:.1f}"
        else:
            s_d = "0"
    if r0 == r1:
        if r0 != 0:
            s_r = f"{r0:.1f}"
        else:
            s_r = "0"
    if p0 == p1:
        if p0 != 0:
            s_p = f"{p0:.1f}"
        else:
            s_p = "0"
    if o0 == o1:
        if o0 != 0:
            s_o = f"{o0:.1f}"
        else:
            s_o = "0"

    stringy = [s_d, s_r, s_p, s_o]
    for i in range(3):
        fig.add_annotation(text=stringy[i], x=[0.1, 0.5, 0.9][i], y=reg_i*1.25, showarrow=False, row=2, col=3, font=dict(size=17, color=['grey', col_receive, 'tomato'][i]))

fig.update_xaxes(row=2, col=3, range=(0, 1), showticklabels=False)
fig.update_yaxes(row=2, col=3, range=(-0.5, 15), showticklabels=False)

fig.add_shape(
    col=1, row=2,
    type="line",
    x0=0, x1=0,
    y0=-1e3, y1=1e3,
    line=dict(color="black",  dash='dash',width=3)
)
fig.update_xaxes(row=2, col=1, range=(-1, 3), tickfont=dict(size=16),
                    tickvals=[-1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5],
                    ticktext=['-100%', '-50%', 'No<br>reduction', '50%', '100%', '150%', '200%', '250%'])
fig.update_yaxes(row=2, col=1, range=(-0.5, 15
                                      ), showticklabels=False)

fig.write_image("K:/Code/effort-sharing/Figures/COMMITTEDpolicybrief/Figure_"+str(year_of_focus)+".png", scale=5)
fig.show()